In [25]:
import tensorflow as tf
import numpy as np
import os
import time
import sys

In [26]:
print("Version: ", tf.__version__)
#print("Eager mode: ", tf.executing_eagerly())
print("GPU esta", "disponible" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print("Dispositivos disponibles: ", tf.config.list_physical_devices())

Version:  2.7.0
GPU esta disponible
Dispositivos disponibles:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [27]:
tf.device('/GPU:0')

In [28]:
fileDL= tf.keras.utils.get_file('LasMinasDelReySalomon.txt','https://raw.githubusercontent.com/DuvanAndresVelasquez/DeepLearning/main/ciudad_perros.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')

In [29]:
texto = texto.lower()

In [30]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:27733 caracteres
el texto esta compuesto de estos :50 caracteres
['\n', '\r', ' ', '!', '"', "'", '(', ',', '-', '.', '6', ':', ';', '?', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '¿', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '“', '”']


In [31]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: '-cuatro -dijo el jaguar.\r\nlos rostros se suavizaron en el resplandor vacilante que el globo de luz d'
array([ 8, 17, 34, 15, 33, 31, 28,  2,  8, 18, 23, 24, 28,  2, 19, 25,  2,
       24, 15, 21, 34, 15, 31,  9,  1,  0, 25, 28, 32,  2, 31, 28, 32, 33,
       31, 28, 32,  2, 32, 19,  2, 32, 34, 15, 35, 23, 38, 15, 31, 28, 27,
        2, 19, 27,  2, 19, 25,  2, 31, 19, 32, 29, 25, 15, 27, 18, 28, 31,
        2, 35, 15, 17, 23, 25, 15, 27, 33, 19,  2, 30, 34, 19,  2, 19, 25,
        2, 21, 25, 28, 16, 28,  2, 18, 19,  2, 25, 34, 38,  2, 18])


In [32]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
secu_length=150
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'-cuatro -dijo el jaguar.\r\nlos rostros se suavizaron en el resplandor vacilante que el globo de luz difundía por el recinto, a través\r\nde escasas partíc'
'ulas limpias de vidrio: el peligro había desaparecido para todos, salvo para porfirio\r\ncava. los dados estaban quietos, marcaban tres y uno, su blancur'
'a contrastaba con el suelo sucio.\r\n-cuatro -repitió el jaguar- ¿quién?\r\n-yo -murmuró cava- dije cuatro.\r\n-apúrate -replicó el jaguar- ya sabes, el segu'
'ndo de la izquierda.\r\ncava sintió frío. los baños estaban al fondo de las cuadras, separados de ellas por una delgada puerta de\r\nmadera, y no tenían ve'
'ntanas. en años anteriores, el invierno sólo llegaba al dormitorio de los cadetes,\r\ncolándose por los vidrios rotos y las rendijas; pero este año era a'
'gresivo y casi ningún rincón de] colegio\r\nse libraba del viento, que, en las noches, conseguía penetrar hasta en los baños, disipar la hediondez\r\nacumu'
'lada durante el día y destruir su atmósfera tibia. pe

In [33]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  '-cuatro -dijo el jaguar.\r\nlos rostros se suavizaron en el resplandor vacilante que el globo de luz difundía por el recinto, a través\r\nde escasas partí'
Target data:  'cuatro -dijo el jaguar.\r\nlos rostros se suavizaron en el resplandor vacilante que el globo de luz difundía por el recinto, a través\r\nde escasas partíc'


In [34]:
print(dataset)
BATCH_SIZE= 64 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


In [35]:
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
embedding_dim= 256
rnn_units=1024
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (64, None, 256)           12800     
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_3 (Dense)             (64, None, 50)            51250     
                                                                 
Total params: 5,311,026
Trainable params: 5,311,026
Non-trainable params: 0
_________________________________________________________________


In [36]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [37]:
checkpoint_dir='/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpoints_test/'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")
cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')

In [39]:
EPOCHS=400
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/400
2/2 [==============================] - ETA: 0s - loss: 1.5278 - accuracy: 0.5372
Epoch 00001: loss improved from 1.55035 to 1.52777, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpoints_test/cp_0001.ckpt
2/2 [==============================] - 1s 866ms/step - loss: 1.5278 - accuracy: 0.5372
Epoch 2/400
2/2 [==============================] - ETA: 0s - loss: 1.5137 - accuracy: 0.5494
Epoch 00002: loss improved from 1.52777 to 1.51375, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpoints_test/cp_0002.ckpt
2/2 [==============================] - 1s 633ms/step - loss: 1.5137 - accuracy: 0.5494
Epoch 3/400
2/2 [==============================] - ETA: 0s - loss: 1.4985 - accuracy: 0.5494
Epoch 00003: loss improved from 1.51375 to 1.49850, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (

In [40]:
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpoints_test/cp_0400.ckpt


In [41]:
model.load_weights(latest)
history2=model.fit(dataset, 
                    epochs=50, 
                    verbose=1,
                    callbacks=[cp_callback])

Epoch 1/50
2/2 [==============================] - ETA: 0s - loss: 0.0510 - accuracy: 0.9868
Epoch 00001: loss did not improve from 0.04562
2/2 [==============================] - 3s 274ms/step - loss: 0.0510 - accuracy: 0.9868
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 0.0449 - accuracy: 0.9887
Epoch 00002: loss improved from 0.04562 to 0.04491, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpoints_test/cp_0002.ckpt
2/2 [==============================] - 1s 829ms/step - loss: 0.0449 - accuracy: 0.9887
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 0.0482 - accuracy: 0.9878
Epoch 00003: loss did not improve from 0.04491
2/2 [==============================] - 1s 236ms/step - loss: 0.0482 - accuracy: 0.9878
Epoch 4/50
2/2 [==============================] - ETA: 0s - loss: 0.0484 - accuracy: 0.9877
Epoch 00004: loss did not improve from 0.04491
2/2 [==============================]

In [43]:
LOG_DIR = 'tb_logs'
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-11-23 03:05:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.19M  5.81MB/s    in 2.3s    

2021-11-23 03:05:18 (5.81 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [44]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

In [45]:
history_TB=model.fit(dataset, 
                    epochs=200, 
                    verbose=1,
                    callbacks=[tbCallBack])

Epoch 1/200
2/2 [==============================] - 3s 3s/step - loss: 0.0415 - accuracy: 0.9893
Epoch 2/200
2/2 [==============================] - 3s 2s/step - loss: 0.0413 - accuracy: 0.9895
Epoch 3/200
2/2 [==============================] - 3s 2s/step - loss: 0.0443 - accuracy: 0.9885
Epoch 4/200
2/2 [==============================] - 3s 2s/step - loss: 0.0432 - accuracy: 0.9886
Epoch 5/200
2/2 [==============================] - 3s 2s/step - loss: 0.0432 - accuracy: 0.9890
Epoch 6/200
2/2 [==============================] - 3s 2s/step - loss: 0.0430 - accuracy: 0.9885
Epoch 7/200
2/2 [==============================] - 3s 2s/step - loss: 0.0407 - accuracy: 0.9896
Epoch 8/200
2/2 [==============================] - 3s 2s/step - loss: 0.0435 - accuracy: 0.9889
Epoch 9/200
2/2 [==============================] - 3s 2s/step - loss: 0.0449 - accuracy: 0.9887
Epoch 10/200
2/2 [==============================] - 3s 2s/step - loss: 0.0422 - accuracy: 0.9893
Epoch 11/200
2/2 [=====================

In [46]:
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [47]:
def generate_text(model, start_string):
  num_generate=500
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []
  temperature = 0.2 
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

In [54]:
print(generate_text(model, start_string=u"alterada instantáneamente por ronquidos"))

alterada instantáneamente por ronquidos de lo sier quinta año de la
estatua del héroe. hace dos años, la distribución de las cuadras era distinta; los cadetes de quinto
ocupaban las cuadras vecinas a lucidez
insólita.
comenzó a avanzar pegado a la pared. en vez de cruzar el patio, dio un rodeo, siguiendo el muro curvo
de las cuadras de quinto. al baño empujando la puerta con las dos manos. en la claridad
amarillenta del recinto, cava comprobó que el jaguar estaba descalzo; sus pies eran gran las sábanas hasta el cuello, un alie
